In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.firefox.options import Options
import keyboard

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'ft_com_paid'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# functions
def button_wait(xpath):
    e = WebDriverWait(browser, 20).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.click()
    
    
def text_wait(xpath,text):
    e = WebDriverWait(browser, 20).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.send_keys(text)
    
    
def text_wait_enter(xpath,text):
    e = WebDriverWait(browser, 20).until(ec.visibility_of_element_located((By.XPATH, xpath)))
    e.send_keys(text)
    e.send_keys(Keys.ENTER)


# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_fetch_page_url = 0
publish_source = 'ft.com'
country = 'United Kingdom'
language = 'English'
images_path = []


options = Options()
options.headless = True
# browser = webdriver.Firefox(options=options)
browser = webdriver.Chrome(executable_path="chromedriver.exe")
browser.maximize_window()

try:
    browser.get('https://accounts.ft.com/login')
except:
    logger.info('Unable to fetch login page\n')
    browser.quit()
        

text_wait('//*[@id="enter-email"]', 'swendlrow@gmail.com')
time.sleep(2)
button_wait('//*[@id="enter-email-next"]')
time.sleep(5)
text_wait('//*[@id="enter-password"]', 'Takesubs@2021')
time.sleep(2)
button_wait('//*[@id="sign-in-button"]')
time.sleep(2)

# button_wait('//*[@id="social-login"]')
# time.sleep(5)
# text_wait('//input[ @type="email" or @class="whsOnd zHQkBf"]', 'swendlrow@gmail.com')
# time.sleep(3)
# button_wait("//div[@id='identifierNext']")
# time.sleep(5)
# text_wait('//input[ @type="password" or @class="whsOnd zHQkBf"]', 'redlion@MMXX')
# time.sleep(3)
# button_wait("//div[@id='passwordNext']")
# time.sleep(5)



keyboard.wait('esc')

for i in range(1,7):
    try:
        logger.info(f'Rss_page  https://www.ft.com/news-feed?page={i}\n')
        browser.get(f'https://www.ft.com/news-feed?page={i}')
    except:
        logger.info('Unable to fetch home page\n')

    time.sleep(random.randint(1,3))
    rss_feed = browser.page_source
    for _ in rss_feed.split('_heading"><')[1:]:  

        source_link ="https://www.ft.com"+c.scrap('\s*href="(.*?)"', _)  

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        logger.info(f'Fetching link {source_link}\n\n')
        try:
            browser.get(source_link)
        except:
            logger.info('Unable to fetch home page\n')
        page = browser.page_source

        if page.startswith('Unable to fetch'):
            logger.info(page) # writes error message with error code
            unable_to_fetch_page_url += 1
            continue           


        # source_headline
        source_headline = c.scrap('<meta\s*property="og:title"\s*content="(.*?)"', page)
        if not source_headline:
            source_headline = c.scrap('<blockquote>(.*?)</blockquote>', page)

        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

        # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('datePublished":"(.*?)"', page).split('.',-1)[0]   #datetime="2021-06-30T04:00:00Z"
            if not date_time_str:
                date_time_str = c.scrap('datetime="(.*?)"', page) 
            date_time_str = re.sub('Z','',date_time_str)
            date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S')
            ist_date_time = date_time_obj + timedelta(hours = 5,minutes = 30)  # utc time to ist
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            # print(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        #if pub_date != created_on:
        #     break    
 
        # source_content          
        source_content = c.scrap('content-body">(.*?)<script class="js-magnet-data', page)
        if not source_content:
            source_content = c.scrap('articleBody":"(.*?)","wordCount', page)
        if not source_content:
            source_content =  source_headline           
        source_content = re.sub('<aside.*?>(.*?)</aside>', "", source_content, flags = re.S)
        source_content = re.sub('<figcaption.*?>(.*?)</figcaption>', "", source_content, flags = re.S)    
        source_content = re.sub('<figure.*?>(.*?)</figure>', "", source_content, flags = re.S)
        source_content = c.strip_html(source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist = c.scrap('<meta\s*property="article:author"\s*content="(.*?)"', page)
        if not journalist : journalist = 'NA' 

        # # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  # get object id and insert data
        no_of_data += 1

browser.quit()
exit()

logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch url: {unable_to_fetch_page_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')